# [병원 개/폐업 분류 예측 경진대회(데이콘)](https://dacon.io/competitions/official/9565/overview/description/)

In [56]:
import numpy as np
import pandas as pd

In [57]:
# colab으로 실행할 때
# from google.colab import drive
# drive.mount('/content/drive')
# train_df = pd.read_csv('/content/drive/MyDrive/kaggle-study/1/data02/train.csv')
# test_df = pd.read_csv('/content/drive/MyDrive/kaggle-study/1/data02/test.csv')
# submission_df = pd.read_csv('/content/drive/MyDrive/kaggle-study/1/data02/submission_sample.csv')

# jupyter로 실행할 때
train_df = pd.read_csv('./data02/train.csv')
test_df = pd.read_csv('./data02/test.csv')
submission_df = pd.read_csv('./data02/submission_sample.csv')

In [58]:
train_df.head()

,inst_id,OC,sido,sgg,openDate,bedCount,instkind,revenue1,salescost1,sga1,...,debt2,liquidLiabilities2,shortLoan2,NCLiabilities2,longLoan2,netAsset2,surplus2,employee1,employee2,ownerChange
0,1,open,choongnam,73,20071228,175.0,nursing_hospital,4.217530e+09,0.0,3.961135e+09,...,7.589937e+08,2.228769e+08,0.000000e+00,5.361169e+08,3.900000e+08,2.619290e+09,1.271224e+09,62.0,64.0,same
1,3,open,gyeongnam,32,19970401,410.0,general_hospital,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,801.0,813.0,same
2,4,open,gyeonggi,89,20161228,468.0,nursing_hospital,1.004522e+09,515483669.0,4.472197e+08,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,234.0,1.0,same
3,7,open,incheon,141,20000814,353.0,general_hospital,7.250734e+10,0.0,7.067740e+10,...,3.775501e+10,1.701860e+10,9.219427e+09,2.073641e+10,1.510000e+10,1.295427e+10,7.740829e+09,663.0,663.0,same
4,9,open,gyeongnam,32,20050901,196.0,general_hospital,4.904354e+10,0.0,4.765605e+10,...,5.143259e+10,3.007259e+10,1.759375e+10,2.136001e+10,1.410803e+10,5.561941e+06,9.025550e+09,206.0,197.0,same


In [59]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 58 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   inst_id             301 non-null    int64  
 1   OC                  301 non-null    object 
 2   sido                301 non-null    object 
 3   sgg                 301 non-null    int64  
 4   openDate            301 non-null    int64  
 5   bedCount            296 non-null    float64
 6   instkind            300 non-null    object 
 7   revenue1            293 non-null    float64
 8   salescost1          293 non-null    float64
 9   sga1                293 non-null    float64
 10  salary1             293 non-null    float64
 11  noi1                293 non-null    float64
 12  noe1                293 non-null    float64
 13  interest1           293 non-null    float64
 14  ctax1               293 non-null    float64
 15  profit1             293 non-null    float64
 16  liquidAs

train.csv - 의료기관이 폐업했는지 여부를 포함하여 최근 2개년의 재무정보와 병원 기본정보 
test.csv - 폐업 여부를 제외하고 train.csv와 동일 

sample_submission.csv - inst_id와 open과 close를 예측하는 OC 두개의 열로 구성. OC의 값은 open 예측일 경우 1, close 예측일 경우 0.



inst_id - 각 파일에서의 병원 고유 번호

OC – 영업/폐업 분류, 2018년 폐업은 2017년 폐업으로 간주함

sido – 병원의 광역 지역 정보

sgg – 병원의 시군구 자료

openDate – 병원 설립일

bedCount - 병원이 갖추고 있는 병상의 수

instkind – 병원, 의원, 요양병원, 한의원, 종합병원 등 병원의 종류

·        종합병원 : 입원환자 100명 이상 수용 가능

·        병원 : 입원 환자 30명 이상 100명 미만 수용 가능

·        의원 : 입원 환자 30명 이하 수용 가능

·        한방 병원(한의원) : 침술과 한약으로 치료하는 의료 기관.  

 

revenue1 – 매출액, 2017(회계년도)년 데이터를 의미함

salescost1 – 매출원가, 2017(회계년도)년 데이터를 의미함

sga1 - 판매비와 관리비, 2017(회계년도)년 데이터를 의미함

salary1 – 급여, 2017(회계년도)년 데이터를 의미함

noi1 – 영업외수익, 2017(회계년도)년 데이터를 의미함

noe1 – 영업외비용, 2017(회계년도)년 데이터를 의미함

Interest1 – 이자비용, 2017(회계년도)년 데이터를 의미함

ctax1 – 법인세비용, 2017(회계년도)년 데이터를 의미함

Profit1 – 당기순이익, 2017(회계년도)년 데이터를 의미함

liquidAsset1 – 유동자산, 2017(회계년도)년 데이터를 의미함

quickAsset1 – 당좌자산, 2017(회계년도)년 데이터를 의미함

receivableS1 - 미수금(단기), 2017(회계년도)년 데이터를 의미함

inventoryAsset1 – 재고자산, 2017(회계년도)년 데이터를 의미함

nonCAsset1 – 비유동자산, 2017(회계년도)년 데이터를 의미함

tanAsset1 – 유형자산, 2017(회계년도)년 데이터를 의미함

OnonCAsset1 - 기타 비유동자산, 2017(회계년도)년 데이터를 의미함

receivableL1 – 장기미수금, 2017(회계년도)년 데이터를 의미함

debt1 – 부채총계, 2017(회계년도)년 데이터를 의미함

liquidLiabilities1 – 유동부채, 2017(회계년도)년 데이터를 의미함

shortLoan1 – 단기차입금, 2017(회계년도)년 데이터를 의미함

NCLiabilities1 – 비유동부채, 2017(회계년도)년 데이터를 의미함

longLoan1 – 장기차입금, 2017(회계년도)년 데이터를 의미함

netAsset1 – 순자산총계, 2017(회계년도)년 데이터를 의미함

surplus1 – 이익잉여금, 2017(회계년도)년 데이터를 의미함

 

revenue2 – 매출액, 2016(회계년도)년 데이터를 의미함

salescost2 – 매출원가, 2016(회계년도)년 데이터를 의미함

sga2 - 판매비와 관리비, 2016(회계년도)년 데이터를 의미함

salary2 – 급여, 2016(회계년도)년 데이터를 의미함

noi2 – 영업외수익, 2016(회계년도)년 데이터를 의미함

noe2 – 영업외비용, 2016(회계년도)년 데이터를 의미함

interest2 – 이자비용, 2016(회계년도)년 데이터를 의미함

ctax2 – 법인세비용, 2016(회계년도)년 데이터를 의미함

profit2 – 당기순이익, 2016(회계년도)년 데이터를 의미함

liquidAsset2 – 유동자산, 2016(회계년도)년 데이터를 의미함

quickAsset2 – 당좌자산, 2016(회계년도)년 데이터를 의미함

receivableS2 - 미수금(단기), 2016(회계년도)년 데이터를 의미함

inventoryAsset2 – 재고자산, 2016(회계년도)년 데이터를 의미함

nonCAsset2 – 비유동자산, 2016(회계년도)년 데이터를 의미함

tanAsset2 – 유형자산, 2016(회계년도)년 데이터를 의미함

OnonCAsset2 - 기타 비유동자산, 2016(회계년도)년 데이터를 의미함

receivableL2 – 장기미수금, 2016(회계년도)년 데이터를 의미함

Debt2 – 부채총계, 2016(회계년도)년 데이터를 의미함

liquidLiabilities2 – 유동부채, 2016(회계년도)년 데이터를 의미함

shortLoan2 – 단기차입금, 2016(회계년도)년 데이터를 의미함

NCLiabilities2 – 비유동부채, 2016(회계년도)년 데이터를 의미함

longLoan2 – 장기차입금, 2016(회계년도)년 데이터를 의미함

netAsset2 – 순자산총계, 2016(회계년도)년 데이터를 의미함

surplus2 – 이익잉여금, 2016(회계년도)년 데이터를 의미함

employee1 – 고용한 총 직원의 수, 2017(회계년도)년 데이터를 의미함

employee2 – 고용한 총 직원의 수, 2016(회계년도)년 데이터를 의미함

ownerChange – 대표자의 변동 

# 데이터 전처리

In [60]:
train_df['OC'].value_counts()

open      286
 close     15
Name: OC, dtype: int64

In [61]:
from sklearn.preprocessing import OrdinalEncoder

# fit_transform 의 인자로 trian[['OC']]를 넣는다. train_df['OC'] X 
# DataFrame이 반환되냐, Series가 반환되냐 의 차이
type(train_df['OC'])
type(train_df[['OC']])
OC_encoder = OrdinalEncoder()
OC_encoded = OC_encoder.fit_transform(train_df[['OC']])
train_df['OC'] = OC_encoded

In [62]:
train_df['OC'].value_counts()

1.0    286
0.0     15
Name: OC, dtype: int64

In [63]:
# 피처와 타겟값 분리
y_train = train_df['OC']
X_train = train_df.drop('OC', axis=1)

X_test = test_df.drop('OC', axis=1)

In [64]:
# 식별을 위한 id 값이므로 삭제
X_train = X_train.drop('inst_id', axis=1)

In [65]:
# ownerChange와 instkind 가장 많은 값으로 대체

X_train['ownerChange'][X_train['ownerChange'].isnull()] = X_train['ownerChange'].value_counts().index[0]
X_train['instkind'][X_train['instkind'].isnull()] = X_train['instkind'].value_counts().index[0]

C:\Users\yrc92\.conda\envs\ai_study_py37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\yrc92\.conda\envs\ai_study_py37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [76]:
# ownerChange 숫자 인코딩

ownerChange_encoder = OrdinalEncoder()
X_train['ownerChange'] = ownerChange_encoder.fit_transform(X_train[['ownerChange']])
X_train['ownerChange'].value_counts()

1.0    259
0.0     42
Name: ownerChange, dtype: int64

In [67]:
def split_openDate_to_ymd(X):
    X['openDate'] = X['openDate'].astype(str)
    X['openYear'] = X['openDate'].str[:4]
    X['openMonth'] = X['openDate'].str[4:6]
    X['openDay'] = X['openDate'].str[6:]
    
    X = X.drop('openDate', axis=1)
    return X

In [68]:
X_train = split_openDate_to_ymd(X_train)
X_test = split_openDate_to_ymd(X_test)

In [69]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 58 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   sido                301 non-null    object 
 1   sgg                 301 non-null    int64  
 2   bedCount            296 non-null    float64
 3   instkind            301 non-null    object 
 4   revenue1            293 non-null    float64
 5   salescost1          293 non-null    float64
 6   sga1                293 non-null    float64
 7   salary1             293 non-null    float64
 8   noi1                293 non-null    float64
 9   noe1                293 non-null    float64
 10  interest1           293 non-null    float64
 11  ctax1               293 non-null    float64
 12  profit1             293 non-null    float64
 13  liquidAsset1        293 non-null    float64
 14  quickAsset1         293 non-null    float64
 15  receivableS1        293 non-null    float64
 16  inventor

In [70]:
from sklearn.preprocessing import OneHotEncoder

In [71]:
def preprocess_categorical(ohencoder, X_train, X_test, colname):
    data_all = np.concatenate([X_train[colname], X_test[colname]])
    ohencoder.fit(data_all.reshape(-1, 1))
    processed = ohencoder.transform(X_train[[colname]])
    return processed

In [72]:
sido_encoder = OneHotEncoder()
sgg_encoder = OneHotEncoder()
instkind_encoder = OneHotEncoder()
openYear_encoder = OneHotEncoder()
openMonth_encoder = OneHotEncoder()
openDay_encoder = OneHotEncoder()

X_train['sido'] = preprocess_categorical(sido_encoder, X_train, X_test, 'sido')
X_train['sgg'] = preprocess_categorical(sgg_encoder, X_train, X_test, 'sgg')
X_train['instkind'] = preprocess_categorical(instkind_encoder, X_train, X_test, 'instkind')
X_train['openYear'] = preprocess_categorical(openYear_encoder, X_train, X_test, 'openYear')
X_train['openMonth'] = preprocess_categorical(openMonth_encoder, X_train, X_test, 'openMonth')
X_train['openDay'] = preprocess_categorical(openDay_encoder, X_train, X_test, 'openDay')

0        (0, 2)\t1.0\n  (1, 9)\t1.0\n  (2, 8)\t1.0\n ...
1        (0, 2)\t1.0\n  (1, 9)\t1.0\n  (2, 8)\t1.0\n ...
2        (0, 2)\t1.0\n  (1, 9)\t1.0\n  (2, 8)\t1.0\n ...
3        (0, 2)\t1.0\n  (1, 9)\t1.0\n  (2, 8)\t1.0\n ...
4        (0, 2)\t1.0\n  (1, 9)\t1.0\n  (2, 8)\t1.0\n ...
                             ...                        
296      (0, 2)\t1.0\n  (1, 9)\t1.0\n  (2, 8)\t1.0\n ...
297      (0, 2)\t1.0\n  (1, 9)\t1.0\n  (2, 8)\t1.0\n ...
298      (0, 2)\t1.0\n  (1, 9)\t1.0\n  (2, 8)\t1.0\n ...
299      (0, 2)\t1.0\n  (1, 9)\t1.0\n  (2, 8)\t1.0\n ...
300      (0, 2)\t1.0\n  (1, 9)\t1.0\n  (2, 8)\t1.0\n ...
Name: sido, Length: 301, dtype: object

In [78]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 58 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   sido                301 non-null    object 
 1   sgg                 301 non-null    object 
 2   bedCount            296 non-null    float64
 3   instkind            301 non-null    object 
 4   revenue1            293 non-null    float64
 5   salescost1          293 non-null    float64
 6   sga1                293 non-null    float64
 7   salary1             293 non-null    float64
 8   noi1                293 non-null    float64
 9   noe1                293 non-null    float64
 10  interest1           293 non-null    float64
 11  ctax1               293 non-null    float64
 12  profit1             293 non-null    float64
 13  liquidAsset1        293 non-null    float64
 14  quickAsset1         293 non-null    float64
 15  receivableS1        293 non-null    float64
 16  inventor

In [79]:
cat_attribs = [
  'sido', 'sgg', 'instkind', 'ownerChange', 'openYear', 'openMonth', 'openDay'
]

In [91]:
X_train_cat = X_train[cat_attribs]
X_train_cat.shape

(301, 7)

In [92]:
X_train_num = X_train.drop(cat_attribs, axis=1)
X_train_num.shape

(301, 51)

In [93]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')
imputer.fit(X_train_num)
X_train_num = imputer.transform(X_train_num)

In [97]:
X_train_preprocessed = np.concatenate([X_train_cat, X_train_num], axis=1)

In [98]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rnd_clf = RandomForestClassifier(n_jobs=-1)
accuracy = cross_val_score(rnd_clf, X_train_preprocessed, y_train, scoring='accuracy')

C:\Users\yrc92\.conda\envs\ai_study_py37\lib\site-packages\sklearn\model_selection\_validation.py:614: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
TypeError: float() argument must be a string or a number, not 'csr_matrix'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\yrc92\.conda\envs\ai_study_py37\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\yrc92\.conda\envs\ai_study_py37\lib\site-packages\sklearn\ensemble\_forest.py", line 305, in fit
    accept_sparse="csc", dtype=DTYPE)
  File "C:\Users\yrc92\.conda\envs\ai_study_py37\lib\site-packages\sklearn\base.py", line 433, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "C:\Users\yrc92\.conda\envs\ai_study_py37\lib\site-packages\sklearn\utils\validation.

In [99]:
accuracy

array([nan, nan, nan, nan, nan])